In [2]:
import os

import requests
from bs4 import BeautifulSoup

In [3]:
def is_a_valid_recipe(soup):
    title = soup.title.string
    ingredients = soup.find('section', 'recipe-ingredients')
    instructions = soup.find('section', 'recipe-directions')
    result = title != 'Allrecipes - File Not Found' and bool(ingredients) and bool(instructions)
    return result

In [4]:
total_urls  = 0
total_recipes = 0
recipe_errors = 0
response_errors = 0

base_url = 'http://allrecipes.com/recipe/'
lower_limit = 6663
upper_limit = 246200
# for ide in range(lower_limit, upper_limit + 1):
for ide in range(6901, 6906):
    try:
        str_id = str(ide)
        folder_path = 'data/allrecipes/' + str(ide // 1000 * 1000)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        file_path = folder_path + '/' + str_id + '.html'
        if not os.path.isfile(file_path):
            url = base_url + str_id
            response = requests.get(url)
            if response.ok:
                soup = BeautifulSoup(response.content, 'html.parser')
                if is_a_valid_recipe(soup):
                    with open(file_path, 'w') as f:
                        f.write(soup.prettify())
                        total_recipes += 1
                else:
                    recipe_errors += 1
            else:
                response_errors += 1
    except:
        pass
    total_urls += 1
    if total_urls % 1000 == 0:
        print(total_urls, 'processed.')

print('Total urls:', total_urls)
print('Total recipes:', total_recipes)
print('Recipe errors:', recipe_errors)
print('Response errors:', response_errors)

Total urls: 5
Total recipes: 1
Recipe errors: 0
Response errors: 0


In [15]:
url = 'http://allrecipes.com/recipe/139620'
url = 'http://allrecipes.com/recipe//8919'
r = requests.get(url, headers={'referer': 'http://allrecipes.com/recipes/231/world-cuisine/european/'})
soup = BeautifulSoup(r.content, 'html.parser')

In [16]:
with open('test.html', 'w') as f:
    f.write(soup.prettify())

In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <!-- ARLOG SERVER: USSEAMOBILE16 LOCAL_IP: 192.168.5.142 MERCH_KEY: MerchData_4_2_427_139620_CA_68 -->
 <head>
  <title>
   Avocado and Tuna Tapas Recipe - Allrecipes.com
  </title>
  <meta content="Avocado and Tuna Tapas Recipe" property="og:title"/>
  <meta content="Allrecipes" property="og:site_name"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Avocado halves are filled with a tangy tuna salad in this Spanish tapa. This is a light, healthy tapa that goes best with crisp white wines and crunchy bread." id="metaDescription" name="description">
   <meta content="Avocado halves are filled with a tangy tuna salad in this Spanish tapa. This is a light, healthy tapa that goes best with crisp white wines and crunchy bread." property="og:description"/>
   <meta content="noodp,noydir" name="robots"/>
   <link href="http://allrecipes.com/recipe/139620/avocado-and-tuna-tapas/" id="canonicalUrl" rel